# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install ../input/medspacy/medspacy-0.2.0.0-py3-none-any.whl
import os
os.system('python -m pip install --no-index --find-links=../input/medspcay/medspacy medspacy')#install Medspcay

Medspcay
MedSpaCy is a library of tools for performing clinical NLP and text processing tasks with the popular spaCy framework. The medspacy package brings together a number of other packages, each of which implements specific functionality for common clinical text processing specific to the clinical domain, such as sentence segmentation, contextual analysis and attribute assertion, and section detection.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import ast
import numpy as np
import random
import nltk.data
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
##############
import spacy
import medspacy
from medspacy.ner import TargetRule
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent,visualize_dep
from medspacy.custom_tokenizer import create_medspacy_tokenizer
from medspacy.sentence_splitting import PyRuSHSentencizer
from medspacy.context import ConTextComponent, ConTextRule
from medspacy.section_detection import Sectionizer
from medspacy.io import DocConsumer
from medspacy.ner import TargetMatcher, TargetRule
from spacy import displacy
from IPython.core.display import display, HTML
# import spacy_transformers
from spacy.tokens import Span



In [ ]:
#Read Data
train = pd.read_csv("../input/nbme-score-clinical-patient-notes/train.csv")
test = pd.read_csv("../input/nbme-score-clinical-patient-notes/test.csv")
features = pd.read_csv("../input/nbme-score-clinical-patient-notes/features.csv")
patient_notes = pd.read_csv("../input/nbme-score-clinical-patient-notes/patient_notes.csv")
submission = pd.read_csv("../input/nbme-score-clinical-patient-notes/sample_submission.csv")

In [ ]:
patient_notes

In [ ]:
#Merge Data
train = train.merge(features, on=['feature_num', 'case_num'], how='left')
train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(train.head())

In [ ]:
for idx in range(len(train)): 
    train["location"][idx] = train["location"][idx].replace(";" , "','")  
     # For replacing semicolon ";"  with comma  ","

train["location"] = train["location"].apply(eval)  # Converting to list

In [ ]:
# Thanks @Y.NAKAMA
#https://www.kaggle.com/code/yasufuminakama/nbme-deberta-base-baseline-train

# incorrect annotation
train.at[338, 'annotation'] = ast.literal_eval('["father heart attack"]')
train.at[338, 'location'] = ast.literal_eval('["764 783"]')

train.at[621, 'annotation'] = ast.literal_eval('["for the last 2-3 months"]')
train.at[621, 'location'] = ast.literal_eval('["77 100"]')

train.at[655, 'annotation'] = ast.literal_eval('["no heat intolerance", "no cold intolerance"]')
train.at[655, 'location'] = ast.literal_eval('["285 292","301 312", "285 287","296 312"]')

train.at[1262, 'annotation'] = ast.literal_eval('["mother thyroid problem"]')
train.at[1262, 'location'] = ast.literal_eval('["551 557","565 580"]')

train.at[1265, 'annotation'] = ast.literal_eval('[\'felt like he was going to "pass out"\']')
train.at[1265, 'location'] = ast.literal_eval('["131 135","181 212"]')

train.at[1396, 'annotation'] = ast.literal_eval('["stool , with no blood"]')
train.at[1396, 'location'] = ast.literal_eval('["259 280"]')

train.at[1591, 'annotation'] = ast.literal_eval('["diarrhoe non blooody"]')
train.at[1591, 'location'] = ast.literal_eval('["176 184","201 212"]')

train.at[1615, 'annotation'] = ast.literal_eval('["diarrhea for last 2-3 days"]')
train.at[1615, 'location'] = ast.literal_eval('["249 257","271 288"]')

train.at[1664, 'annotation'] = ast.literal_eval('["no vaginal discharge"]')
train.at[1664, 'location'] = ast.literal_eval('["822 824","907 924"]')

train.at[1714, 'annotation'] = ast.literal_eval('["started about 8-10 hours ago"]')
train.at[1714, 'location'] = ast.literal_eval('["101 129"]')

train.at[1929, 'annotation'] = ast.literal_eval('["no blood in the stool"]')
train.at[1929, 'location'] = ast.literal_eval('["531 539","549 561"]')

train.at[2134, 'annotation'] = ast.literal_eval('["last sexually active 9 months ago"]')
train.at[2134, 'location'] = ast.literal_eval('["540 560","581 593"]')

train.at[2191, 'annotation'] = ast.literal_eval('["right lower quadrant pain"]')
train.at[2191, 'location'] = ast.literal_eval('["32 57"]')

train.at[2553, 'annotation'] = ast.literal_eval('["diarrhoea no blood"]')
train.at[2553, 'location'] = ast.literal_eval('["308 317","376 384"]')

train.at[3124, 'annotation'] = ast.literal_eval('["sweating"]')
train.at[3124, 'location'] = ast.literal_eval('["549 557"]')

train.at[3858, 'annotation'] = ast.literal_eval('["previously as regular", "previously eveyr 28-29 days", "previously lasting 5 days", "previously regular flow"]')
train.at[3858, 'location'] = ast.literal_eval('["102 123", "102 112","125 141", "102 112","143 157", "102 112","159 171"]')

train.at[4373, 'annotation'] = ast.literal_eval('["for 2 months"]')
train.at[4373, 'location'] = ast.literal_eval('["33 45"]')

train.at[4763, 'annotation'] = ast.literal_eval('["35 year old"]')
train.at[4763, 'location'] = ast.literal_eval('["5 16"]')

train.at[4782, 'annotation'] = ast.literal_eval('["darker brown stools"]')
train.at[4782, 'location'] = ast.literal_eval('["175 194"]')

train.at[4908, 'annotation'] = ast.literal_eval('["uncle with peptic ulcer"]')
train.at[4908, 'location'] = ast.literal_eval('["700 723"]')

train.at[6016, 'annotation'] = ast.literal_eval('["difficulty falling asleep"]')
train.at[6016, 'location'] = ast.literal_eval('["225 250"]')

train.at[6192, 'annotation'] = ast.literal_eval('["helps to take care of aging mother and in-laws"]')
train.at[6192, 'location'] = ast.literal_eval('["197 218","236 260"]')

train.at[6380, 'annotation'] = ast.literal_eval('["No hair changes", "No skin changes", "No GI changes", "No palpitations", "No excessive sweating"]')
train.at[6380, 'location'] = ast.literal_eval('["480 482","507 519", "480 482","499 503","512 519", "480 482","521 531", "480 482","533 545", "480 482","564 582"]')

train.at[6562, 'annotation'] = ast.literal_eval('["stressed due to taking care of her mother", "stressed due to taking care of husbands parents"]')
train.at[6562, 'location'] = ast.literal_eval('["290 320","327 337", "290 320","342 358"]')

train.at[6862, 'annotation'] = ast.literal_eval('["stressor taking care of many sick family members"]')
train.at[6862, 'location'] = ast.literal_eval('["288 296","324 363"]')

train.at[7022, 'annotation'] = ast.literal_eval('["heart started racing and felt numbness for the 1st time in her finger tips"]')
train.at[7022, 'location'] = ast.literal_eval('["108 182"]')

train.at[7422, 'annotation'] = ast.literal_eval('["first started 5 yrs"]')
train.at[7422, 'location'] = ast.literal_eval('["102 121"]')

train.at[8876, 'annotation'] = ast.literal_eval('["No shortness of breath"]')
train.at[8876, 'location'] = ast.literal_eval('["481 483","533 552"]')

train.at[9027, 'annotation'] = ast.literal_eval('["recent URI", "nasal stuffines, rhinorrhea, for 3-4 days"]')
train.at[9027, 'location'] = ast.literal_eval('["92 102", "123 164"]')

train.at[9938, 'annotation'] = ast.literal_eval('["irregularity with her cycles", "heavier bleeding", "changes her pad every couple hours"]')
train.at[9938, 'location'] = ast.literal_eval('["89 117", "122 138", "368 402"]')

train.at[9973, 'annotation'] = ast.literal_eval('["gaining 10-15 lbs"]')
train.at[9973, 'location'] = ast.literal_eval('["344 361"]')

train.at[10513, 'annotation'] = ast.literal_eval('["weight gain", "gain of 10-16lbs"]')
train.at[10513, 'location'] = ast.literal_eval('["600 611", "607 623"]')

train.at[11551, 'annotation'] = ast.literal_eval('["seeing her son knows are not real"]')
train.at[11551, 'location'] = ast.literal_eval('["386 400","443 461"]')

train.at[11677, 'annotation'] = ast.literal_eval('["saw him once in the kitchen after he died"]')
train.at[11677, 'location'] = ast.literal_eval('["160 201"]')

train.at[12124, 'annotation'] = ast.literal_eval('["tried Ambien but it didnt work"]')
train.at[12124, 'location'] = ast.literal_eval('["325 337","349 366"]')

train.at[12279, 'annotation'] = ast.literal_eval('["heard what she described as a party later than evening these things did not actually happen"]')
train.at[12279, 'location'] = ast.literal_eval('["405 459","488 524"]')

train.at[12289, 'annotation'] = ast.literal_eval('["experienced seeing her son at the kitchen table these things did not actually happen"]')
train.at[12289, 'location'] = ast.literal_eval('["353 400","488 524"]')

train.at[13238, 'annotation'] = ast.literal_eval('["SCRACHY THROAT", "RUNNY NOSE"]')
train.at[13238, 'location'] = ast.literal_eval('["293 307", "321 331"]')

train.at[13297, 'annotation'] = ast.literal_eval('["without improvement when taking tylenol", "without improvement when taking ibuprofen"]')
train.at[13297, 'location'] = ast.literal_eval('["182 221","182 213","225 234"]')

train.at[13299, 'annotation'] = ast.literal_eval('["yesterday", "yesterday"]')
train.at[13299, 'location'] = ast.literal_eval('["79 88", "409 418"]')

train.at[13845, 'annotation'] = ast.literal_eval('["headache global", "headache throughout her head"]')
train.at[13845, 'location'] = ast.literal_eval('["86 94","230 236", "86 94","237 256"]')

train.at[14083, 'annotation'] = ast.literal_eval('["headache generalized in her head"]')
train.at[14083, 'location'] = ast.literal_eval('["56 64","156 179"]')

In [ ]:
#extract clinical concepts and attributes from text
target_rules={}
for i in range(10):
  target_rules[i]=[]

In [ ]:
from spacy.tokens import Span
Span.set_extension("case_num", default="",force=True)
Span.set_extension("feature_num", default="",force=True)
for idx in range(len(train)):
  note=train["pn_history"][idx]
  case_num=train["case_num"][idx]
  feature_num=train["feature_num"][idx]
  loc= train["location"][idx]
  for pos in loc: 
      start_pos=(int(pos.split()[0]))  #Starting position of annotation
      end_pos=(int(pos.split()[1]))  #Ending position of annotation
      match=note[start_pos:end_pos]
      # print(match)
      target_rule=TargetRule(match,'NER',attributes={"case_num": case_num,'feature_num':feature_num})
      target_rules[case_num].append(target_rule)

In [ ]:
#Target Rules Example
target_rules[0]

In [ ]:
#Example
text=train.loc[0,'pn_history']
target_rule=target_rules[train.loc[0,'case_num']]

In [ ]:
nlp = medspacy.load(enable=['pyrush', 'target_matcher', 'context', 'sectionizer'])
nlp.get_pipe("medspacy_target_matcher").add(target_rule)
doc = nlp(text)

In [ ]:
doc_consumer = DocConsumer(nlp, dtypes=("ent", "context", "section", "doc"))
doc = doc_consumer(doc)

In [ ]:
doc_consumer.dtype_attrs

In [ ]:
#Convert TO Data Frame
doc._.to_dataframe("ent").head(10)

In [ ]:
data = test.copy()
for row in data.index:
    case_num = data.loc[row, 'case_num']
    pn_num = data.loc[row, 'pn_num']
    feature_num = data.loc[row, 'feature_num']
    
    
    # Extract Patient Note
    text = patient_notes[(patient_notes['pn_num']==pn_num)&(patient_notes['case_num']==case_num)]['pn_history'].values[0]
    target_rule=target_rules[case_num]
    nlp = medspacy.load(enable=['pyrush', 'target_matcher', 'context', 'sectionizer'])
    nlp.get_pipe("medspacy_target_matcher").add(target_rule)
    doc = nlp(text)
    pred = ''
    for ent in doc.ents:
        if (ent._.case_num==case_num)&(ent._.feature_num	==feature_num):
            pred += str(ent.start_char)+' '+str(ent.end_char)+';'
    pred = pred[:-1]
    data.loc[row, 'location'] = pred
    submission.loc[row, 'id'] = data.loc[row, 'id']
    submission.loc[row, 'location'] = pred
submission[['id', 'location']].to_csv('submission.csv', index=False)   